In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

sys.path.append(os.getcwd() + "/scripts/")
sys.path.append(os.getcwd())

from run_experiment import (generate_explanations, calculate_ensembles, get_filenames,
                            print_image, calculate_statistics, compute_score_table)
from statistics_record import AttrbutionsRecord
from sklearn.neural_network import BernoulliRBM

In [2]:
title = "mnist_test"
dataset = "mnist"
ensemble_name = "mnist_notebook_test"

In [3]:
generate_explanations(title, dataset, ["DeepLift", "IntegratedGradient", "Saliency"], 10)


Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



...

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

simple_model saved
Finished training


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking arugment for argument weight in method wrapper_thnn_conv2d_forward)

In [ ]:
files = get_filenames(title, "individual_methods")
ar1 = AttrbutionsRecord.load(title, "individual_attribution", files[0])

image_id = 1
print_image(ar1.images[image_id].cpu().numpy(), "Original image")

for i, attr in enumerate(ar1.attributions[image_id]):
    print_image(attr, ar1.methods[i])

In [ ]:
task1 = dict({
    "title": "mean",
    "technique": "mean",
    "individual_methods": [1, 1, 1],
    "nr_noise": 0,
    "rbm": None,
})

task2 = dict({
    "title": "rbm",
    "technique": "rbm",
    "individual_methods": [1, 1, 1],
    "nr_noise": 0,
    "rbm": [BernoulliRBM(n_components=1, batch_size=10, learning_rate=0.01, n_iter=100)]
})

calculate_ensembles(title, ensemble_name, [task1, task2], True)

In [ ]:
files = get_filenames(title, ensemble_name)
ar1 = AttrbutionsRecord.load(title, ensemble_name, files[0])

image_id = 1

for i, (attr, ensemble) in enumerate(ar1.get_ensemble_double_rbm(image_id)):
    print_image(attr, ensemble)

In [ ]:
calculate_statistics(title, ensemble_name, True, True, True)

In [ ]:
df = compute_score_table(title, ensemble_name)
df